In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path



In [ ]:


def rename_and_drop_columns(input_df, column_mapping_df):
    # Create a dictionary from the column_mapping dataframe
    column_mapping_dict = column_mapping_df.set_index('file_column')['rpm_column'].to_dict()
    
    # Rename the columns in the manual dataframe
    input_df = input_df.rename(columns=column_mapping_dict)
    
    # Find the columns in the manual dataframe that are not in the column_mapping dataframe
    columns_to_drop = [col for col in input_df.columns if col not in column_mapping_dict.values()]
    
    # Drop the columns that are not in the column_mapping dataframe
    input_df = input_df.drop(columns=columns_to_drop)
    
    return input_df

def basic_validation(df1, df2, result_file):
    #Remove 'Id' column and 'VIA' Column if any exists
    if (any(column_name == 'id' for column_name in df2.columns)): 
        df2.drop(['id'], axis=1, inplace=True)
    if (any(column_name == 'VIA' for column_name in df1.columns)): 
        df1.drop(['VIA'], axis=1, inplace=True)

    # Check if the dataframes have the same number of rows and columns
    if df1.shape == df2.shape:
        result_file.write("The dataframes have the same number of rows and columns.\n")
    else:
        result_file.write("The dataframes do not have the same number of rows and columns.\n")
        result_file.write(f"The manual dataframe has {df1.shape[0]} rows and {df1.shape[1]} columns.\n")
        result_file.write(f"The automated dataframe has {df2.shape[0]} rows and {df2.shape[1]} columns.\n")
        print (f"The manual dataframe has {df1.shape[0]} rows and {df1.shape[1]} columns.\n")
        print (f"The automated dataframe has {df2.shape[0]} rows and {df2.shape[1]} columns.\n")
        return pd.DataFrame(), pd.DataFrame()
    
    # Check if the dataframes have the same columns
    if (compare_columns_results := compare_columns(df1, df2, result_file)!=0):
        print (compare_columns_results)
        return pd.empty, pd.empty
    else:
        return df1, df2

def sort_rows(unsorted_df, columns_to_sort):
    # Check if all columns in columns_to_sort exist in the dataframe
    if all(item in unsorted_df.columns for item in columns_to_sort):
        # Sort the dataframe
        df_sorted = unsorted_df.sort_values(list(columns_to_sort)).reset_index(drop=True)
        return df_sorted
    else:
        return "One or more columns in columns_to_sort do not exist in the dataframe."

def compare_columns(df1, df2, result_file):
    if set(df1.columns) == set(df2.columns):
        # Check if corresponding columns in both dataframes have the same data type
        for column in df1.columns:
            if df1[column].dtype != df2[column].dtype:
                print(df1)
                result_file.write(f"The column '{column}' has different data types in the dataframes.\n")
                print(f"The column '{column}' has different data types in the dataframes.")
                return f"The column '{column}' has different data types in the dataframes."
            else:
                result_file.write("The dataframes have the same columns and data types.\n")
                return 0
    else:
        different_columns = list(set(df1.columns) ^ set(df2.columns))
        result_file.write(f"The dataframes have different columns: {different_columns}\n")  
        return f"The dataframes have different columns: {different_columns}"

def compare_df(df1, df2, result_file):
    if set(df1.columns) == set(df2.columns):
 
        # Select only numeric columns
        df1_numeric = df1.select_dtypes(include=[np.number]).reset_index(drop=True)
        df2_numeric = df2.select_dtypes(include=[np.number]).reset_index(drop=True)
        
        # Create a boolean mask for values in df1_numeric and df2_numeric that are not close
        mask = ~np.isclose(df1_numeric.values, df2_numeric.values, atol=0.01)
        
        # Apply the mask to df1_numeric and df2_numeric, and drop rows that contain only NaN
        diff_numeric = df1_numeric.where(mask).compare(df2_numeric.where(mask))
        
        # Select only non-numeric columns
        df1_non_numeric = df1.select_dtypes(exclude=[np.number])
        df2_non_numeric = df2.select_dtypes(exclude=[np.number])
        
        # Compare non-numeric columns
        diff_non_numeric = df1_non_numeric.compare(df2_non_numeric)
        
        # Concatenate the differences
        diff = pd.concat([diff_numeric, diff_non_numeric])
        
        # Reset the index and add the old index as a new column
        diff = diff.reset_index().rename(columns={'index': 'original_row_number', 'self': 'manual', 'other': 'automated'})
        
        if not diff.empty:
            result_file.write(f"The dataframes have differences.\n")
            return diff
        else:
            result_file.write(f"The dataframes have the same columns and values.\n")
            return "The dataframes have the same columns and values."
    else:
        different_columns = list(set(df1.columns) ^ set(df2.columns))
        return f"The dataframes have different columns: {different_columns}"



In [2]:
# def main():    
# vendor = input ("Vendor: ")
vendor = 'Appsmart'

# filename = input ("Filename (mmm-yy): ")
filename = 'oct-23'

# Set variable to 1 to start a new results file, or to 0 to append to an existing file
new_file = 1

filenamea = filename + 'a.xlsx'
filenamem = filename + 'm.xlsx'
columns_to_sort = ('Account', 'Product name', 'Net billed', 'Gross commission')

# establish file paths for input and output files
manual_url = Path(vendor, "test_files", "rpm_files_manual", filenamem)
automated_url = Path(vendor, "test_files", "rpm_files_automation", filenamea)
output_url = Path(vendor, "test_files", "output", filename + '_output.xlsx')
column_mapping_url = Path(vendor, "test_files", "column_mapping.xlsx")
test_results_url = Path(vendor, "test_files", "output", "test_results.txt")



# read in the input files
try:
    manual = pd.read_excel(manual_url)
except:
    print ("Error opening manual file")

try:
    automated = pd.read_excel(automated_url)
except:
    print ("ERROR: opening automated file")

try:
    column_mapping = pd.read_excel(column_mapping_url)
except:
    print ("ERROR: opening mapping file")

try:
    if(new_file):
        test_results = open(test_results_url, 'wt')
        test_results.write  (f"Test results for '{vendor}'\n")
    else:
        test_results = open(test_results_url, 'at')
except:
    print ("ERROR: opening test results file")

test_results.write  (f"\n\n{'-'*37}\nTest results for '{vendor}' - '{filename}' \n")

# run basic validation on the dataframes
manual, automated = basic_validation(manual, automated, test_results)

if (manual.empty or automated.empty):
    print ("ERROR: Dataframes failed basic validation")
    test_results.write  ("ERROR: Dataframes failed basic validation")
else:
    # Remap columns in manual and automated based on the mapping dataframe
    #  this is done to ensure that all files are sorted and compared in the same way
    manual = rename_and_drop_columns(manual, column_mapping)
    automated = rename_and_drop_columns(automated, column_mapping)

    # Sort columns on both dataframes to compare
    manual = sort_rows(manual, columns_to_sort)
    automated = sort_rows(automated, columns_to_sort)

    # Compare the sorted dataframes        
    df_differences = compare_df(manual, automated, result_file=test_results)
    print(df_differences)

    # Produce output files
    with pd.ExcelWriter(output_url) as writer:
        manual.to_excel(writer, sheet_name='manual', index=False)
        automated.to_excel(writer, sheet_name='automated', index=False)
    #     if isinstance(df_differences, pd.DataFrame):
    #         df_differences.to_excel(writer, sheet_name='differences', index=False)
    #     else:
    #         pd.DataFrame([df_differences]).to_excel(writer, sheet_name='differences', index=False)

    print ("ERROR: Dataframes failed basic validation")
    test_results.write  ("ERROR: Dataframes failed basic validation")
test_results.close()


NameError: name 'Path' is not defined

In [ ]:
# # Run the main function
# main()